In [1]:
# pip install geopandas matplotlib pillow

In [92]:
# import os
# import geopandas as gpd
# import matplotlib.pyplot as plt
# from PIL import Image
# import io
# from tqdm import tqdm

# def process_json_file(json_file, output_folder):
#     # JSON 파일 불러오기
#     gdf = gpd.read_file(json_file)

#     # 이미지 크기 설정
#     image_size = (512, 512)

#     # 폴리곤을 하나의 이미지로 합치기
#     fig, ax = plt.subplots(figsize=(5, 5))
#     fig.patch.set_facecolor('white')  # 배경색을 흰색으로 설정

#     for idx, row in gdf.iterrows():
#         # 각 폴리곤을 'ANN_CD' 값에 따라 다른 그레이스케일 톤으로 채움
#         gray_value = row['ANN_CD'] / 255
#         gdf[gdf.index == idx].plot(ax=ax, color=(gray_value, gray_value, gray_value))

#     # 축 제거 및 이미지 크기 조정
#     plt.axis('off')
#     plt.xlim(gdf.total_bounds[0], gdf.total_bounds[2])
#     plt.ylim(gdf.total_bounds[1], gdf.total_bounds[3])
#     ax.set_aspect('equal', adjustable='box')

#     # 이미지로 변환
#     buf = io.BytesIO()
#     plt.savefig(buf, format='png', bbox_inches='tight', pad_inches=0, facecolor=fig.get_facecolor(), edgecolor='none')
#     buf.seek(0)
#     img = Image.open(buf)
#     img = img.resize(image_size, Image.Resampling.LANCZOS)
#     buf.close()
#     plt.close(fig)

#     # 이미지 파일 이름 설정 (JSON 파일 이름 기반)
#     image_file_name = os.path.splitext(os.path.basename(json_file))[0] + '.png'
#     image_path = os.path.join(output_folder, image_file_name)

#     # 이미지 저장
#     img.save(image_path)

In [134]:
import cv2
import numpy as np
import geopandas as gpd
import os
from tqdm import tqdm

def process_json_file(json_file, output_folder):
    # JSON 파일 불러오기
    gdf = gpd.read_file(json_file)
    
    # 이미지 크기 설정
    image_size = (512, 512)
    
    # 빈 이미지 생성 (모든 픽셀을 흰색으로 설정)
    img = np.ones((image_size[0], image_size[1], 3), dtype=np.uint8) * 255
    
    # 좌표 변환을 위한 최소, 최대 좌표 계산
    x_min, y_min, x_max, y_max = gdf.total_bounds
    x_range = x_max - x_min
    y_range = y_max - y_min
    
    # 'ANN_CD' 필드의 최대값 설정
    max_ann_cd = 255
    class_mapping = {
    210: 160,
    220: 170,
    230: 180
    }

    for idx, row in gdf.iterrows():
        # 'ANN_CD' 값 변환
        if row['ANN_CD'] in class_mapping:
            ann_cd_transformed = class_mapping[row['ANN_CD']]
        else:
            ann_cd_transformed = row['ANN_CD']
        # 각 폴리곤의 'ANN_CD' 값이 190을 초과하는 경우 190으로 설정
        ann_cd = min(row['ANN_CD'], max_ann_cd)

        color = (ann_cd, ann_cd, ann_cd)  # 그레이스케일
        
        # 폴리곤의 좌표를 이미지의 픽셀 좌표로 변환
        coordinates = row.geometry.exterior.coords.xy
        x_coordinates = (np.array(coordinates[0]) - x_min) / x_range * image_size[0]
        y_coordinates = image_size[1] - (np.array(coordinates[1]) - y_min) / y_range * image_size[1]  # y축 방향 반전
        polygon = np.array([list(zip(x_coordinates, y_coordinates))], dtype=np.int32)
        
        # 폴리곤의 영역을 그레이스케일 톤으로 채우기
        cv2.fillPoly(img, polygon, color)
    
    # 이미지 파일 이름 설정 (JSON 파일 이름 기반)
    image_file_name = os.path.splitext(os.path.basename(json_file))[0] + '.png'
    image_path = os.path.join(output_folder, image_file_name)
    
    # 이미지 저장
    cv2.imwrite(image_path, img)
    
    # # 이미지 화면에 표시
    # cv2.imshow('Image', img)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()


In [135]:
# 폴더 경로 설정
folder_path = ['Dataset\Training\label\AP10_City_Json','Dataset\Training\label\AP10_Forest_Json','Dataset\Training\label\SN10_Forest_Json','Dataset\Training\label\AP25_City_Json','Dataset\Training\label\AP25_Forest_Json',
               'Dataset\Validation\label\AP10_City_Json','Dataset\Validation\label\AP10_Forest_Json','Dataset\Validation\label\AP25_City_Json','Dataset\Validation\label\AP25_Forest_Json','Dataset\Validation\label\SN10_Forest_Json']

for path in folder_path:
    print(path)
    output_folder = path.replace('Json', 'GT')
    # output_folder가 존재하지 않으면 생성
    if not os.path.exists(output_folder):
        os.makedirs(output_folder, exist_ok=True)
    # 폴더 내의 모든 JSON 파일에 대해 작업 수행
    for file in tqdm(os.listdir(path)):
        if file.endswith('.json'):
            json_file_path = os.path.join(path, file)
            process_json_file(json_file_path, output_folder)


Dataset\Training\label\AP10_City_Json


100%|██████████| 3200/3200 [00:37<00:00, 84.93it/s] 


Dataset\Training\label\AP10_Forest_Json


100%|██████████| 12800/12800 [02:25<00:00, 87.87it/s] 


Dataset\Training\label\SN10_Forest_Json


...

100%|██████████| 400/400 [00:22<00:00, 17.91it/s]


Dataset\Validation\label\AP25_Forest_Json


100%|██████████| 1600/1600 [00:31<00:00, 51.45it/s]


Dataset\Validation\label\SN10_Forest_Json


100%|██████████| 100/100 [00:11<00:00,  8.64it/s]


In [136]:
import cv2
image = cv2.imread("C:\Workspace\CarbonCapturePredict\Dataset\Validation\label\AP25_Forest_GT\AP_25_37604057_0179.png", cv2.IMREAD_GRAYSCALE)
# 픽셀 값 확인
print(image)


[[130 130 130 ... 140 140 140]
 [130 130 130 ... 140 140 140]
 [130 130 130 ... 140 140 140]
 ...
 [130 130 130 ... 140 140 140]
 [130 130 130 ... 140 140 140]
 [130 130 130 ... 140 140 140]]
